In [38]:
test_mode = False

In [39]:
import pandas as pd
import hopsworks
from utils import *
import json
import os
import warnings
from dotenv import load_dotenv
import datetime
#from datetime import timezone

warnings.filterwarnings("ignore")

In [40]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")


project = hopsworks.login()

2025-01-04 16:08:34,168 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-04 16:08:34,173 INFO: Initializing external client
2025-01-04 16:08:34,173 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-04 16:08:38,199 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [41]:

fs = project.get_feature_store() 
#secrets = secrets_api(project.name)

# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60


today


1735999721

In [42]:

# Step 2: Retrieve the feature group by name and version
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=1
)

# Step 3: Read the data from the feature group
s_data_df = solana_feature_group.select(["timestamp"]).read()

# Step 4: Find the maximum date
solana_max_date = s_data_df['timestamp'].max()
solana_difference = (today - solana_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {solana_max_date}")
print(f"Difference in days: {solana_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.95s) 
The maximum date is: 1735948800
Difference in days: 0


In [43]:
# Step 2: Retrieve the feature group by name and version
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=1
)

# Step 3: Read the data from the feature group
b_data_df = bitcoin_feature_group.select(["timestamp"]).read()

# Step 4: Find the maximum date
bitcoin_max_date = b_data_df['timestamp'].max()
bitcoin_difference = (today - bitcoin_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {bitcoin_max_date}")
print(f"Difference in days: {bitcoin_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.23s) 
The maximum date is: 1735862400
Difference in days: 1


In [44]:
#fs = project.get_feature_store(name='kth_lab1_2024_featurestore')
fs = project.get_feature_store()

fng_feature_group = fs.get_feature_group(
    name='f_n_g_index', 
    version=1)
fng_df = fng_feature_group.read()

# Step 4: Find the maximum date
fng_max_date = int(fng_df['timestamp'].max())
fng_difference = (today - fng_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {fng_max_date}")
print(f"Difference in days: {fng_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.67s) from Hopsworks, using Hopsworks Feature Query Service.   Reading data from Hopsworks, using Hopsworks Feature Query Service...   Reading data from Hopsworks, using Hopsworks Feature Query Service.   Reading data from Hopsworks, using Hopsworks Feature Query Service...   Reading data from Hopsworks, using Hopsworks Feature Query Service.   Reading data from Hopsworks, using Hopsworks Feature Query Service...   Reading data from Hopsworks, using Hopsworks Feature Query Service.   Reading data from Hopsworks, using Hopsworks Feature Query Service...   
The maximum date is: 1735862400
Difference in days: 1


### Getting missing data

### Request solana price from last available data till now

In [45]:

url = "https://data-api.cryptocompare.com//spot/v1/historical/days"

# Load environment variables from the .env file
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


params = {
    "market":"kraken",
    "instrument": "SOL-USD",  
    "limit": solana_difference,  
    "api_key": crypto_api_key,
}
if solana_difference!=0:
    sol_response = trigger_request(url, params)
    sol_response_df = pd.DataFrame(sol_response["Data"])
    sol_response_df = sol_response_df[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
    sol_response_df.columns = sol_response_df.columns.str.lower()
    sol_response_df['open'] = sol_response_df['open'].astype(float)
    sol_response_df['close'] = sol_response_df['close'].astype(float)
    sol_response_df['high'] = sol_response_df['high'].astype(float)
    sol_response_df['low'] = sol_response_df['low'].astype(float)
    sol_response_df['volume'] = sol_response_df['volume'].astype(float)
    sol_response_df['volume_buy'] = sol_response_df['volume_buy'].astype(float)



    print("Num entries to add: ", len(sol_response_df))
else:
    sol_response_df = pd.DataFrame(columns=["timestamp","open","high", "low","close", "volume", "volume_buy", "volume_sell"])

In [46]:
sol_response_df.head()

,timestamp,open,high,low,close,volume,volume_buy,volume_sell


### Request bitcoin price from last available data till now

In [47]:
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


params = {
    "market":"kraken",
    "instrument": "BTC-USD",  
    "limit": bitcoin_difference,  
    "api_key": crypto_api_key,
}

if bitcoin_difference>0:
    bitcoin_response = trigger_request(url, params)
    bitcoin_response_df = pd.DataFrame(bitcoin_response["Data"])
    bitcoin_response_df = bitcoin_response_df[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
    bitcoin_response_df.columns = bitcoin_response_df.columns.str.lower()
    bitcoin_response_df['open'] = bitcoin_response_df['open'].astype(float)
    bitcoin_response_df['close'] = bitcoin_response_df['close'].astype(float)
    bitcoin_response_df['high'] = bitcoin_response_df['high'].astype(float)
    bitcoin_response_df['low'] = bitcoin_response_df['low'].astype(float)
    bitcoin_response_df['volume'] = bitcoin_response_df['volume'].astype(float)
    bitcoin_response_df['volume_buy'] = bitcoin_response_df['volume_buy'].astype(float)

    print("Num entries to add: ", len(bitcoin_response_df))
else:
    bitcoin_response_df = pd.DataFrame(columns=["timestamp","open","high", "low","close", "volume", "volume_buy", "volume_sell"])

bitcoin_response_df.head()

Num entries to add:  1


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1735948800,98111.1,98231.0,97548.4,97958.0,353.865464,230.754242,123.111222


In [48]:
bitcoin_response_df

,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1735948800,98111.1,98231.0,97548.4,97958.0,353.865464,230.754242,123.111222


### Request fear and greed index values from last available data till now

In [49]:
url = "https://api.alternative.me/fng/?"

# Set the parameters for the API request
params = {
    "limit": fng_difference  # Number of days of data to retrieve
}

if fng_difference>0:
    fng_response = trigger_request(url, params)
    fng_df = pd.DataFrame(fng_response['data'])
    fng_df['timestamp'] = fng_df['timestamp'].astype(float)
    fng_df.rename(columns={'value_classification': 'value_classification'}, inplace=True)
    fng_df.rename(columns={'value': 'value'}, inplace=True)

    fng_df.drop(columns=["time_until_update"],inplace=True)
    fng_df.head()
else:
    fng_df = pd.DataFrame(columns=["value","value_classification","timestamp"])
fng_df.head()

,value,value_classification,timestamp
0,73,Greed,1.735949e+09


### Now we need to use the responses to update the data in Hopsworks

In [50]:
if not test_mode and sol_response_df.shape[0]>0:
    solana_feature_group.insert(sol_response_df)



In [51]:
if not test_mode and bitcoin_response_df.shape[0]>0:
    bitcoin_feature_group.insert(bitcoin_response_df)


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: bitcoin_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/bitcoin_1_offline_fg_materialization/executions


In [52]:
fng_df.head()

,value,value_classification,timestamp
0,73,Greed,1.735949e+09


In [53]:
if not test_mode and fng_df.shape[0]>0:
    fng_feature_group.insert(fng_df)

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: f_n_g_index_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/f_n_g_index_1_offline_fg_materialization/executions
